In [47]:
from __future__ import division
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [48]:
import os

cwd = os.getcwd()

parent = os.path.dirname(cwd)

parent

'/Users/juju/Library/Mobile Documents/com~apple~CloudDocs/ta_project'

In [49]:
city_name = "Miami"

# Import review data and STR data

In [50]:
review_data = pd.read_excel(os.path.join(parent+r"/data/monthly_review_data",r"{}_monthly_data.xlsx".format(city_name)))

In [51]:
review_data = review_data[review_data["post_month"]<'2017-01']

In [52]:
review_data["SHARE ID"].nunique()

161

326

In [46]:
review_data["SHARE ID"].nunique()

128

In [29]:
review_data["monthly_review_count"].sum()

161521

In [47]:
review_data = pd.read_csv(os.path.join(parent+r"/data",r"tripadvisor_reviews.csv"))

In [48]:
str_data = pd.read_csv(os.path.join(parent+r"/data/cleaned_str_data",r"{}_STR_2022.csv"),\
                       sep=',',header=0,infer_datetime_format=True,\
                       parse_dates=['year_month'])


In [49]:
str_data["year_month"].min()

Timestamp('2000-01-01 00:00:00')

# Format data

In [50]:
str_data["SHARE ID"] = str_data["SHARE ID"].astype(int)

In [51]:
str_data

,year_month,SHARE ID,Year Month,Occ,ADR,RevPAR,Market,Operation,Class,Location,Open Date,month_with_revenue
0,2022-05-01,51622,202205.0,65.3763,179.237458,117.178897,"Nashville, TN",Chain,Upper Midscale,Urban,2022.0,True
1,2022-06-01,51622,202206.0,81.6825,169.175884,138.187159,"Nashville, TN",Chain,Upper Midscale,Urban,2022.0,True
2,2022-07-01,51622,202207.0,79.1705,165.909926,131.351730,"Nashville, TN",Chain,Upper Midscale,Urban,2022.0,True
3,2022-08-01,51622,202208.0,92.2273,174.633604,161.059932,"Nashville, TN",Chain,Upper Midscale,Urban,2022.0,True
4,2022-09-01,51622,202209.0,88.6349,169.458188,150.199130,"Nashville, TN",Chain,Upper Midscale,Urban,2022.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...
214985,2022-08-01,28650555,202208.0,70.8714,180.239130,127.738084,"Miami, FL",Independent,Upper Upscale,Urban,2021.0,True
214986,2022-09-01,28650555,202209.0,61.5422,170.759095,105.089055,"Miami, FL",Independent,Upper Upscale,Urban,2021.0,True
214987,2022-10-01,28650555,202210.0,60.8101,197.896707,120.341375,"Miami, FL",Independent,Upper Upscale,Urban,2021.0,True
214988,2022-11-01,28650555,202211.0,58.2222,230.335115,134.106222,"Miami, FL",Independent,Upper Upscale,Urban,2021.0,True


In [52]:
review_data["SHARE ID"]= review_data["SHARE ID"].astype(int)

In [53]:
#str_data["on_ta"] = str_data["SHARE ID"].isin(review_data["SHARE ID"])

In [54]:
str_data["year_month"] = str_data["year_month"].dt.to_period('M')
review_data["year_month"] = pd.to_datetime(review_data['post_month'], errors='coerce').dt.to_period('M')


In [55]:
str_data["SHARE ID"].nunique()

1416

In [56]:
max_str_month = str_data.groupby("SHARE ID")["year_month"].max().\
                 reset_index().rename(columns={"year_month":"max_str_month"})
min_str_month = str_data.groupby("SHARE ID")["year_month"].min().\
                 reset_index().rename(columns={"year_month":"min_str_month"})

max_review_month = review_data.groupby("SHARE ID")["year_month"].max().\
                 reset_index().rename(columns={"year_month":"max_review_month"})
min_review_month = review_data.groupby("SHARE ID")["year_month"].min().\
                 reset_index().rename(columns={"year_month":"min_review_month"})

In [57]:
is_reviewed =  max_str_month.merge(min_str_month, on="SHARE ID", how="left").\
               merge(max_review_month, on="SHARE ID", how="left").\
               merge(min_review_month, on="SHARE ID",how="left")

In [58]:

is_reviewed["on_ta"] = (is_reviewed["min_review_month"]<=is_reviewed["max_str_month"])&\
                    (is_reviewed["max_review_month"].isnull()==False)


In [59]:
is_reviewed["SHARE ID"].nunique()

1416

In [60]:
str_data = str_data.merge(is_reviewed, on="SHARE ID")
#str_data[str_data["SHARE ID"]==89917]

In [61]:
#str_data[str_data["SHARE ID"]==88627]

In [62]:
review_data[review_data["SHARE ID"]==88627]

,post_month,monthly_rating_mean,monthly_review_count,monthly_one_star_count,monthly_two_star_count,monthly_three_star_count,monthly_four_star_count,monthly_five_star_count,monthly_hotel_response_count,month_with_reviews,num_of_reviews,total_one_star,total_two_star,total_three_star,total_four_star,total_five_star,accum_rating,SHARE ID,year_month
15026,2017-04,5.0,2,0,0,0,0,2,0,True,2,0,0,0,0,2,5.000000,88627,2017-04
15027,2017-05,4.0,2,0,0,0,2,0,0,True,4,0,0,0,2,2,4.500000,88627,2017-05
15028,2017-06,0.0,0,0,0,0,0,0,0,False,4,0,0,0,2,2,4.500000,88627,2017-06
15029,2017-07,5.0,4,0,0,0,0,4,0,True,8,0,0,0,2,6,4.750000,88627,2017-07
15030,2017-08,5.0,4,0,0,0,0,4,0,True,12,0,0,0,2,10,4.833333,88627,2017-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15083,2022-01,0.0,0,0,0,0,0,0,0,False,78,4,10,6,12,46,4.102564,88627,2022-01
15084,2022-02,0.0,0,0,0,0,0,0,0,False,78,4,10,6,12,46,4.102564,88627,2022-02
15085,2022-03,0.0,0,0,0,0,0,0,0,False,78,4,10,6,12,46,4.102564,88627,2022-03
15086,2022-04,0.0,0,0,0,0,0,0,0,False,78,4,10,6,12,46,4.102564,88627,2022-04


In [63]:
#is_reviewed[is_reviewed["SHARE ID"]==88627]["max_review_month"].isnull()

In [64]:
review_data.columns

Index(['post_month', 'monthly_rating_mean', 'monthly_review_count',
       'monthly_one_star_count', 'monthly_two_star_count',
       'monthly_three_star_count', 'monthly_four_star_count',
       'monthly_five_star_count', 'monthly_hotel_response_count',
       'month_with_reviews', 'num_of_reviews', 'total_one_star',
       'total_two_star', 'total_three_star', 'total_four_star',
       'total_five_star', 'accum_rating', 'SHARE ID', 'year_month'],
      dtype='object')

# Merge review data and STR data

In [65]:
'''
data = str_data.merge(review_data[['SHARE ID','year_month','monthly_rating_mean', 'monthly_review_count',
       'monthly_one_star_count', 'monthly_two_star_count',
       'monthly_three_star_count', 'monthly_four_star_count',
       'monthly_five_star_count', 'monthly_hotel_response_count',
       'monthly_partnership_reviews_count', 'cleanliness_count',
       'cleanliness_mean', 'location_count', 'location_mean',
       'sleep_quality_count', 'sleep_quality_mean', 'room_quality_count',
       'room_quality_mean', 'service_count', 'service_mean', 'value_count',
       'value_mean', 'renovate', 'solo_count', 'couple_count', 'family_count',
       'business_count', 'month_with_reviews', 'num_of_reviews',
       'total_one_star', 'total_two_star', 'total_three_star',
       'total_four_star', 'total_five_star', 'accum_rating',
       'total_one_star_%', 'total_two_star_%', 'total_three_star_%',
       'total_four_star_%', 'total_five_star_%']], on=["SHARE ID","year_month"], how="left")
       '''

'\ndata = str_data.merge(review_data[[\'SHARE ID\',\'year_month\',\'monthly_rating_mean\', \'monthly_review_count\',\n       \'monthly_one_star_count\', \'monthly_two_star_count\',\n       \'monthly_three_star_count\', \'monthly_four_star_count\',\n       \'monthly_five_star_count\', \'monthly_hotel_response_count\',\n       \'monthly_partnership_reviews_count\', \'cleanliness_count\',\n       \'cleanliness_mean\', \'location_count\', \'location_mean\',\n       \'sleep_quality_count\', \'sleep_quality_mean\', \'room_quality_count\',\n       \'room_quality_mean\', \'service_count\', \'service_mean\', \'value_count\',\n       \'value_mean\', \'renovate\', \'solo_count\', \'couple_count\', \'family_count\',\n       \'business_count\', \'month_with_reviews\', \'num_of_reviews\',\n       \'total_one_star\', \'total_two_star\', \'total_three_star\',\n       \'total_four_star\', \'total_five_star\', \'accum_rating\',\n       \'total_one_star_%\', \'total_two_star_%\', \'total_three_star_%\',\

In [66]:
data = str_data.merge(review_data, on=["SHARE ID","year_month"], how="left")

In [67]:
data["SHARE ID"].nunique()

1416

In [68]:
not_on_ta = data[data["on_ta"]==0]
on_ta = data[data["on_ta"]==1]

In [69]:
on_ta = on_ta[(on_ta["year_month"]<=on_ta["max_review_month"])]

In [70]:
not_on_ta["SHARE ID"].nunique()

461

In [71]:
on_ta["SHARE ID"].nunique()

951

In [72]:
data["SHARE ID"].nunique()

1416

In [73]:
data[data["on_ta"].isna()]

,year_month,SHARE ID,Year Month,Occ,ADR,RevPAR,Market,Operation,Class,Location,...,monthly_five_star_count,monthly_hotel_response_count,month_with_reviews,num_of_reviews,total_one_star,total_two_star,total_three_star,total_four_star,total_five_star,accum_rating


In [74]:
on_ta[on_ta["max_review_month"].isnull()]

,year_month,SHARE ID,Year Month,Occ,ADR,RevPAR,Market,Operation,Class,Location,...,monthly_five_star_count,monthly_hotel_response_count,month_with_reviews,num_of_reviews,total_one_star,total_two_star,total_three_star,total_four_star,total_five_star,accum_rating


In [75]:
data = pd.concat([not_on_ta, on_ta],axis=0)

In [79]:

data["is_reviewed"] = data["year_month"]>=data["min_review_month"]

In [80]:
data[data["year_month"]<data["min_review_month"]]

,year_month,SHARE ID,Year Month,Occ,ADR,RevPAR,Market,Operation,Class,Location,...,monthly_hotel_response_count,month_with_reviews,num_of_reviews,total_one_star,total_two_star,total_three_star,total_four_star,total_five_star,accum_rating,is_reviewed
45473,2007-01,122761,200701.0,30.7258,107.923885,33.160484,"New York, NY",Independent,Economy,Urban,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
45474,2007-02,122761,200702.0,63.9285,122.272346,78.166964,"New York, NY",Independent,Economy,Urban,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
45475,2007-03,122761,200703.0,83.7903,143.781521,120.475000,"New York, NY",Independent,Economy,Urban,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
45476,2007-04,122761,200704.0,73.5833,193.476784,142.366667,"New York, NY",Independent,Economy,Urban,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
45477,2007-05,122761,200705.0,75.4838,204.431624,154.312903,"New York, NY",Independent,Economy,Urban,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214600,2008-05,10182585,200805.0,52.2222,72.994681,38.119444,"Phoenix, AZ",Chain,Upper Midscale,Suburban,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
214601,2008-06,10182585,200806.0,49.9444,69.836485,34.879444,"Phoenix, AZ",Chain,Upper Midscale,Suburban,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
214602,2008-07,10182585,200807.0,41.0215,67.533421,27.703226,"Phoenix, AZ",Chain,Upper Midscale,Suburban,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
214603,2008-08,10182585,200808.0,36.3440,58.822485,21.378495,"Phoenix, AZ",Chain,Upper Midscale,Suburban,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [81]:
# Save data

output_dir = parent+r"/data/merged_demand_review_data"
#data.to_excel(os.path.join(output_dir,r"{}_data.xlsx".format(city_name)), index = False, encoding='utf-8')
data.to_csv(os.path.join(output_dir,r"rdd_data_2022.csv"), index = False)